## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd

## PD Set Options

In [2]:
pd.set_option('display.max_columns',None)

## Data Read-in

In [3]:
df = pd.read_csv('redfin_2023-11-02-10-47-32.csv')
PBC_gf = gpd.read_file('Palm_Beach_County_Boundary.geojson')

## Data Clean

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
df = df.dropna(subset=['SOLD DATE'])

In [6]:
# Define list of desired months (excluding current month)
desired_months = ['October']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [7]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    641
Name: count, dtype: int64
-------
$/SQUARE FEET
False    640
True       1
Name: count, dtype: int64
-------
YEAR BUILT
False    641
Name: count, dtype: int64
-------


In [8]:
sorted_df = df_filtered.sort_values(by='$/SQUARE FEET', ascending=True)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Boca-Raton/19259-Sabal-Lake-Dr-33434/unit-5092/home/185086108


In [9]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [10]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [11]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
637,PAST SALE,October-18-2023,Condo/Co-op,2855 S Garden Dr S #312,Lake Worth,FL,33461.0,40000.0,1.0,1.5,Lake Clarke Gardens Condo 24,894.0,NaN,1971.0,NaN,45.0,704.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/2855-Gard...,Beaches MLS,RX-10835435,N,Y,26.633896,-80.086099
15,PAST SALE,October-16-2023,Condo/Co-op,360 Windsor P,West Palm Beach,FL,33417.0,70000.0,1.0,1.5,Windsor Condos,702.0,NaN,1972.0,NaN,100.0,493.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/360-...,Beaches MLS,RX-10916798,N,Y,26.711773,-80.135404
509,PAST SALE,October-3-2023,Condo/Co-op,283 Sheffield L,West Palm Beach,FL,33417.0,74000.0,1.0,1.0,Sheffield Condos A TO Q,570.0,43560.0,1971.0,NaN,130.0,343.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/283-...,Beaches MLS,RX-10871352,N,Y,26.721138,-80.126058
185,PAST SALE,October-10-2023,Condo/Co-op,236 Canterbury K #236,West Palm Beach,FL,33417.0,75000.0,1.0,1.5,Century Village,646.0,NaN,1973.0,NaN,116.0,480.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/236-...,Beaches MLS,RX-10917217,N,Y,26.720227,-80.132277
157,PAST SALE,October-31-2023,Condo/Co-op,1722 Bridgewood Dr,Boca Raton,FL,33434.0,75000.0,2.0,2.0,Bridgewood Mid-rise Condo,1255.0,NaN,1974.0,NaN,60.0,914.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1722-Brid...,Beaches MLS,RX-10921140,N,Y,26.373547,-80.165386
533,PAST SALE,October-11-2023,Condo/Co-op,55 Northampton C,West Palm Beach,FL,33417.0,77000.0,1.0,1.5,Century Village,684.0,NaN,1972.0,NaN,113.0,438.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/55-N...,Beaches MLS,RX-10896102,N,Y,26.715683,-80.134325
530,PAST SALE,October-25-2023,Condo/Co-op,120 Windsor F #120,West Palm Beach,FL,33417.0,79900.0,1.0,1.0,Windsor Condos,585.0,NaN,1972.0,NaN,137.0,487.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/120-...,Beaches MLS,RX-10880193,N,Y,26.709548,-80.134058
613,PAST SALE,October-20-2023,Condo/Co-op,139 Somerset G,West Palm Beach,FL,33417.0,80000.0,1.0,1.0,Somerset Condos,570.0,43560.0,1972.0,NaN,140.0,420.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/139-...,Beaches MLS,RX-10858166,N,Y,26.714040,-80.129961
526,PAST SALE,October-30-2023,Condo/Co-op,140 Bedford F #140,West Palm Beach,FL,33417.0,80000.0,1.0,1.0,Century Village,570.0,NaN,1971.0,NaN,140.0,405.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/140-...,Beaches MLS,RX-10898692,N,Y,26.710039,-80.125655
218,PAST SALE,October-13-2023,Condo/Co-op,352 Camden O,West Palm Beach,FL,33417.0,80000.0,1.0,1.0,Century Village - Camden Condos,570.0,NaN,1972.0,NaN,140.0,450.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/352-...,Beaches MLS,RX-10916820,N,Y,26.712805,-80.134588


In [14]:
print(df_filtered['URL'].iloc[509])

https://www.redfin.com/FL/West-Palm-Beach/283-Sheffield-L-33417/home/42146470


In [95]:
# # Correct the prices, if needed
# df_filtered.at[43,'$/SQUARE FEET']=(275000/705)

In [15]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [97]:
# # # Corrections, if needed
# df_filtered.at[245,'$/SQUARE FEET']=(210000/480)
# df_filtered.at[383,'$/SQUARE FEET']=(225000/460)
# df_filtered.at[673,'$/SQUARE FEET']=(550000/960)
# df_filtered.at[777,'$/SQUARE FEET']=(275000/697)

In [16]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
637,40000.0,2855 S Garden Dr S #312,Lake Worth,45.0
157,75000.0,1722 Bridgewood Dr,Boca Raton,60.0
557,89000.0,19259 Sabal Lake Dr #5092,Boca Raton,81.0
439,119000.0,1308 Bridgewood Dr,Boca Raton,85.0
15,70000.0,360 Windsor P,West Palm Beach,100.0
507,95000.0,304 Greenbrier B,West Palm Beach,113.0
533,77000.0,55 Northampton C,West Palm Beach,113.0
459,95500.0,210 Oxford 200 Dr,West Palm Beach,114.0
556,95000.0,2920 Lake Osborne Dr #111,Lake Worth Beach,115.0
185,75000.0,236 Canterbury K #236,West Palm Beach,116.0


In [17]:
print(df_filtered.URL.iloc[637])

https://www.redfin.com/FL/Lake-Worth/2855-Garden-Dr-S-33461/unit-312/home/42291183


In [100]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [18]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [19]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [20]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [21]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [22]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [23]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"October 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(PBC_gf,tooltip='Palm Beach County',name='Palm Beach County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [24]:
m.save('index.html')

## Summary Info

In [25]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [26]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [27]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [28]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Palm-Beach/235-Sunrise-Ave-33480/unit-3054/home/42431755


In [29]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
La Clara, 200 Arkona Ct Unit 25g West Palm Beach | Price $8,965,617 | $2,338 psf | Year built: 2023
Least Expensive
Lake Clarke Gardens Condo 24, 2855 S Garden Dr S #312 Lake Worth | Price $40,000 | $45 psf | Year built: 1971
Highest Price Per Square Foot
La Clara, 200 Arkona Ct Unit 25g West Palm Beach | Price $8,965,617 | $2,338 psf | Year built: 2023
Lowest Price Per Square Foot
Lake Clarke Gardens Condo 24, 2855 S Garden Dr S #312 Lake Worth | Price $40,000 | $45 psf | Year built: 1971
Newest
La Clara, 200 Arkona Ct Unit 25g West Palm Beach | Price $8,965,617 | $2,338 psf | Year built: 2023
Oldest
Palm Beach Hotel Condo, 235 Sunrise Ave #3054 Palm Beach | Price $361,785 | $1,335 psf | Year built: 1925


## Time on Market Calculator

In [30]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/Lake-Worth/2855-Garden-Dr-S-33461/unit-312/home/42291183


In [60]:
from datetime import datetime, timedelta

date1 = datetime(2023, 8, 11) ## List (Earlier) date
date2 = datetime(2023, 9, 26) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

46


## Map URL Snagger

In [31]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [32]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_condo_sales_month_ending_oct_2023


## Get Summary Data

In [33]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 641
--------
Total sale price: $319,371,776
--------
Median sale price: $279,000
--------
Max sale price: $8,965,617
--------
Min sale price: $40,000
------------------------------------------------
PSF INFO
Max price per square foot: $2,338
--------
Min price per square foot: $45
--------
Median price per square foot: $239
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1925.0
----------
Average building age: 1984.4071762870515
